# What's this??
Seleniumを使って三井からデータを引っ張ってくる

https://mfhl.mitsui-chintai.co.jp/chintai/index.html

In [1]:
# import libraries
import time
import math
import pandas as pd
import numpy as np
import os
import urllib.request

In [2]:
# import selenium libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
# import original libraries
from utils.python import mitsui

In [4]:
# variables
path = '/Users/ryusukekimura/Google ドライブ/github/demonstration/rpa/chrome-driver-79.0.3945.36/chromedriver'
url = 'https://mfhl.mitsui-chintai.co.jp/chintai/index.html'
timeout = 30

In [5]:
# create output directory
dir_data = './data/mitsui/'
if not os.path.isdir(dir_data):
    os.makedirs(dir_data)
dir_img = './img/mitsui/'
if not os.path.isdir(dir_img):
    os.makedirs(dir_img)

In [6]:
# define output columns
cols =  ['fetch_datetime', 'todofuken', 'shikuchoson', 'bld_name', 'room_name', 'room_madori', 'bld_address', 'access', 'num_rooms',
              'chinryo', 'kanrihi', 'sikikin', 'reikin', 'kibo_kozo', 'menseki_tsubo', 'syunko_nengappi', 'keiyaku_kikan', 'kousinryo',
               'torihiki_taiyo', 'parking', 'hoken', 'kanri_gaisya', 'kagi_koukan', 'kouza_furikae', 'hosyo_gaisya', 'setsubi', 'bikou']

In [7]:
# for each shikuchouson
for i in range(1, 24):
    
    # initialization
    df = pd.DataFrame(index=[], columns=[cols])
    driver = webdriver.Chrome(executable_path=path)
    driver.implicitly_wait(5)
    driver.get(url)
    
    # move to search page
    mitsui.move_to_search_page_from_top_page(driver, timeout)
    
    # get todohuken & shikuchoson
    todofuken = driver.find_element_by_xpath('//*[@id="container"]/div/div[3]/h4[1]/span').text
    row = math.ceil(i / 4)
    col = i - (row - 1) * 4
    shikuchoson = driver.find_element_by_xpath('//*[@id="東京都２３区_section"]/ul[' + str(row) + ']/li[' + str(col) + ']/span[2]/label').text

    print('|-- ' + todofuken + ' ' + shikuchoson + ' ...')
    
    # checkbox
    driver.find_element_by_xpath('//*[@id="(1,' + str(i) + ')"]').click()
    
    # move to search result page
    mitsui.move_to_search_result_page_from_search_page(driver, timeout)
    
    # for each page
    mitsui.wait_driver(driver, timeout, 'container')
    num_pages = int(driver.find_element_by_xpath('//*[@id="container"]/section/div[2]/div[2]/div[1]/span[1]').text)
    for page in range(1, num_pages + 1):
        print('  |-- page ' + str(page) + ' ...')

        for bukken in range(1, 11):
            print('     |-- bukken ' + str(bukken) + ' ...')
            
            # try
            try:
                # move to room page
                mitsui.wait_driver(driver, timeout, 'container')
                driver.find_element_by_xpath('//*[@id="container"]/section/li[' + str(bukken) + ']/div[2]/div[1]/div/h3').click()
                
                # extract room information
                mitsui.wait_driver(driver, timeout, 'container')
                array = mitsui.extract_information(driver, todofuken, shikuchoson, timeout)

                # add array to dataframe
                record = pd.Series(array, index=df.columns)
                df = df.append(record, ignore_index=True)

                # go back to search result pages
                driver.back()
                
            except: 
                print('fail at extracting room information...')
                
        
        # move to next page
        if page != num_pages:
            mitsui.wait_driver(driver, timeout, 'container')
            driver.find_element_by_xpath('//*[@id="container"]/section/div[1]/div/div/div[2]/div[2]/div[4]/ul/li[2]/a').click()

    # output
    file = './data/mitsui/' + todofuken + '_' + shikuchoson + '.csv'
    df.to_csv(file, index=False)
            
    # close driver
    driver.close()

|-- 東京都２３区 足立区 ...
  |-- page 1 ...
     |-- bukken 1 ...
     |-- bukken 2 ...
     |-- bukken 3 ...
     |-- bukken 4 ...
     |-- bukken 5 ...
     |-- bukken 6 ...
     |-- bukken 7 ...
     |-- bukken 8 ...
fail at extracting room information...
     |-- bukken 9 ...
retry
fail at extracting room information...
     |-- bukken 10 ...
retry
fail at extracting room information...
retry


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=60144): Max retries exceeded with url: /session/6c6fd46314ca44975141ca978e8b7d57/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1077bf6d8>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [8]:
file = './data/mitsui/' + todofuken + '_' + shikuchoson + '.csv'
df = pd.read_csv(file)

In [9]:
df

,fetch_datetime,todofuken,shikuchoson,bld_name,room_name,room_madori,bld_address,access,num_rooms,chinryo,...,kousinryo,torihiki_taiyo,parking,hoken,kanri_gaisya,kagi_koukan,kouza_furikae,hosyo_gaisya,setsubi,bikou
0,<built-in method now of type object at 0x10a49...,東京都２３区,足立区,Ｓｕｎｎｙ Ｓｉｄｅ Ｆｏｒｅｓｔ,２０５,1K,東京都足立区綾瀬４丁目３０番１２号,ＪＲ常磐線綾瀬 徒歩9分\n東京メトロ千代田線綾瀬 徒歩9分\n \n \n...,27戸,"92,000円／月",...,新賃料の1ヶ月分,貸主,無,借家人賠償責任保険を付帯した火災保険にご加入いただきます,未定,NaN,100円／月,"35,000円",区分：必須\n保証会社：ﾚｼﾞﾃﾞﾝﾄｱｼｽﾀﾝｽ(株) 他1社\n条件：初回保証委託料(最...,バルコニー／ エアコン／ 浴室乾燥機／ ＢＳ／ＣＳ／ コンロ／キッチン／ オートロック／エレ...
1,<built-in method now of type object at 0x10a49...,東京都２３区,足立区,Ｓｕｎｎｙ Ｓｉｄｅ Ｆｏｒｅｓｔ,２０６,1K,東京都足立区綾瀬４丁目３０番１２号,ＪＲ常磐線綾瀬 徒歩9分\n東京メトロ千代田線綾瀬 徒歩9分\n \n \n...,27戸,"92,000円／月",...,新賃料の1ヶ月分,貸主,無,借家人賠償責任保険を付帯した火災保険にご加入いただきます,未定,NaN,100円／月,"35,000円",区分：必須\n保証会社：ﾚｼﾞﾃﾞﾝﾄｱｼｽﾀﾝｽ(株) 他1社\n条件：初回保証委託料(最...,バルコニー／ エアコン／ 浴室乾燥機／ ＢＳ／ＣＳ／ コンロ／キッチン／ オートロック／エレ...
2,<built-in method now of type object at 0x10a49...,東京都２３区,足立区,Ｓｕｎｎｙ Ｓｉｄｅ Ｆｏｒｅｓｔ,１０５,1K,東京都足立区綾瀬４丁目３０番１２号,ＪＲ常磐線綾瀬 徒歩9分\n東京メトロ千代田線綾瀬 徒歩9分\n \n \n...,27戸,"93,000円／月",...,新賃料の1ヶ月分,貸主,無,借家人賠償責任保険を付帯した火災保険にご加入いただきます,未定,NaN,100円／月,"35,000円",区分：必須\n保証会社：ﾚｼﾞﾃﾞﾝﾄｱｼｽﾀﾝｽ(株) 他1社\n条件：初回保証委託料(最...,バルコニー／ エアコン／ 浴室乾燥機／ ＢＳ／ＣＳ／ コンロ／キッチン／ 専用庭／オートロッ...
3,<built-in method now of type object at 0x10a49...,東京都２３区,足立区,Ｓｕｎｎｙ Ｓｉｄｅ Ｆｏｒｅｓｔ,２０３,1K,東京都足立区綾瀬４丁目３０番１２号,ＪＲ常磐線綾瀬 徒歩9分\n東京メトロ千代田線綾瀬 徒歩9分\n \n \n...,27戸,"93,000円／月",...,新賃料の1ヶ月分,貸主,無,借家人賠償責任保険を付帯した火災保険にご加入いただきます,未定,NaN,100円／月,"35,000円",区分：必須\n保証会社：ﾚｼﾞﾃﾞﾝﾄｱｼｽﾀﾝｽ(株) 他1社\n条件：初回保証委託料(最...,バルコニー／ エアコン／ 浴室乾燥機／ ＢＳ／ＣＳ／ コンロ／キッチン／ オートロック／エレ...
4,<built-in method now of type object at 0x10a49...,東京都２３区,足立区,Ｓｕｎｎｙ Ｓｉｄｅ Ｆｏｒｅｓｔ,２０７,1K,東京都足立区綾瀬４丁目３０番１２号,ＪＲ常磐線綾瀬 徒歩9分\n東京メトロ千代田線綾瀬 徒歩9分\n \n \n...,27戸,"93,000円／月",...,新賃料の1ヶ月分,貸主,無,借家人賠償責任保険を付帯した火災保険にご加入いただきます,未定,NaN,100円／月,"35,000円",区分：必須\n保証会社：ﾚｼﾞﾃﾞﾝﾄｱｼｽﾀﾝｽ(株) 他1社\n条件：初回保証委託料(最...,バルコニー／ エアコン／ 浴室乾燥機／ ＢＳ／ＣＳ／ コンロ／キッチン／ オートロック／エレ...
5,<built-in method now of type object at 0x10a49...,東京都２３区,足立区,Ｓｕｎｎｙ Ｓｉｄｅ Ｆｏｒｅｓｔ,１０３,1K,東京都足立区綾瀬４丁目３０番１２号,ＪＲ常磐線綾瀬 徒歩9分\n東京メトロ千代田線綾瀬 徒歩9分\n \n \n...,27戸,"94,000円／月",...,新賃料の1ヶ月分,貸主,無,借家人賠償責任保険を付帯した火災保険にご加入いただきます,未定,NaN,100円／月,"35,000円",区分：必須\n保証会社：ﾚｼﾞﾃﾞﾝﾄｱｼｽﾀﾝｽ(株) 他1社\n条件：初回保証委託料(最...,バルコニー／ エアコン／ 浴室乾燥機／ ＢＳ／ＣＳ／ コンロ／キッチン／ 専用庭／オートロッ...
6,<built-in method now of type object at 0x10a49...,東京都２３区,足立区,東京Ｈｉｇｈｔｏｗｅｒ,５０１,1LDK+WIC,東京都足立区千住大川町６－１８,東京メトロ日比谷線北千住 徒歩13分\nＪＲ常磐線北千住 徒歩13分\n東京メトロ千代田線北...,－,"116,000円／月",...,新賃料の1ヶ月分,貸主,有,借家人賠償責任保険を付帯した火災保険にご加入いただきます,グローバルコミュニティ株式会社 東京支社,"20,000円",100円／月,"40,000円",区分：必須\n保証会社：ﾚｼﾞﾃﾞﾝﾄｱｼｽﾀﾝｽ(株) 他1社\n条件：初回保証委託料(最...,バルコニー／ エアコン／ 浴室乾燥機／給湯追い焚き／ＢＳ／ＣＳ／ コンロ／キッチン／グリル／...
7,<built-in method now of type object at 0x10a49...,東京都２３区,足立区,東京Ｈｉｇｈｔｏｗｅｒ,５０２,1LDK+WIC,東京都足立区千住大川町６－１８,東京メトロ日比谷線北千住 徒歩13分\nＪＲ常磐線北千住 徒歩13分\n東京メトロ千代田線北...,－,"116,000円／月",...,新賃料の1ヶ月分,貸主,有,借家人賠償責任保険を付帯した火災保険にご加入いただきます,グローバルコミュニティ株式会社 東京支社,"20,000円",100円／月,"40,000円",区分：必須\n保証会社：ﾚｼﾞﾃﾞﾝﾄｱｼｽﾀﾝｽ(株) 他1社\n条件：初回保証委託料(最...,バルコニー／ エアコン／ 浴室乾燥機／給湯追い焚き／ＢＳ／ＣＳ／ コンロ／キッチン／グリル／...
8,<built-in method now of type object at 0x10a49...,東京都２３区,足立区,Ｓｕｎｎｙ Ｓｉｄｅ Ｆｏｒｅｓｔ,２０６,1K,東京都足立区綾瀬４丁目３０番１２号,ＪＲ常磐線綾瀬 徒歩9分\n東京メトロ千代田線綾瀬 徒歩9分\n \n \n...,27戸,"92,000円／月",...,新賃料の1ヶ月分,貸主,無,借家人賠償責任保険を付帯した火災保険にご加入いただきます,未定,NaN,100円／月,"35,000円",区分：必須\n保証会社：ﾚｼﾞﾃﾞﾝﾄｱｼｽﾀﾝｽ(株) 他1社\n条件：初回保証委託料(最...,バルコニー／ エアコン／ 浴室乾燥機／ ＢＳ／ＣＳ／ コンロ／キッチン／ オートロック／エレ...
9,<built-in method now of type object at 0x10a49...,東京都２３区,足立区,Ｓｕｎｎｙ Ｓｉｄｅ Ｆｏｒｅｓｔ,１０５,1K,東京都足立区綾瀬４丁目３０番１２号,ＪＲ常磐線綾瀬 徒歩9分\n東京メトロ千代田線綾瀬 徒歩9分\n \n \n...,27戸,"93,000円／月",...,新賃料の1ヶ月分,貸主,無,借家人賠償責任保険を付帯した火災保険にご加入いただきます,未定,NaN,100円／月,"35,000円",区分：必須\n保証会社：ﾚｼﾞﾃﾞﾝﾄｱｼｽﾀﾝｽ(株) 他1社\n条件：初回保証委託料(最...,バルコニー／ エアコン／ 浴室乾燥機／ ＢＳ／ＣＳ／ コンロ／キッチン／ 専用庭／オートロッ...
